In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os


/home/user/yes/envs/fastcampus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it", device_map="auto")

model.eval()

model.generation_config.max_new_tokens = 2048


Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.14s/it]


In [3]:
input_text='what is your name?'

input_ids= tokenizer(input_text,return_tensors='pt').to('cuda')

In [4]:
input_ids

{'input_ids': tensor([[     2,   5049,    603,    861,   1503, 235336]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [5]:
outputs=model.generate(**input_ids)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
outputs

tensor([[     2,   5049,    603,    861,   1503, 235336,    109,   2926,   1503,
            603, 137061, 235265,    590,   1144,    671,   2174, 235290,  30316,
          16481,  20409, 235265,    107,    108,      1]], device='cuda:0')

In [10]:
print(tokenizer.batch_decode(outputs,skip_special_tokens=True)[0][len(input_text):].lstrip())

My name is Gemma. I am an open-weights AI assistant.



In [11]:
input_text='안녕하세요!'
input_ids = tokenizer(input_text, return_tensors='pt').to('cuda')

In [13]:
input_ids.input_ids

tensor([[     2, 238179, 243415, 204551, 235341]], device='cuda:0')

In [16]:
outputs= model.generate(
    input_ids=input_ids.input_ids,
    return_dict_in_generate=True,
    output_scores=True
)

In [17]:
logits = outputs.scores

In [39]:
len(logits)

414

In [21]:
len(logits[0][0])

256000

In [48]:
one_logits=logits[14][0]

In [49]:
one_logits

tensor([-3.4900,  5.5700, 10.5176,  ...,  9.0562,  4.2493, -3.1589],
       device='cuda:0')

In [50]:
predict_id=torch.argmax(one_logits).item()

In [51]:
predict_id

242860

In [52]:
tokenizer.decode([predict_id])

'옷'

In [43]:
len(outputs.sequences[0])

419

In [45]:
len(input_ids.input_ids[0])

5

In [47]:
print(tokenizer.batch_decode(outputs.sequences,skip_special_tokens=True)[0])

안녕하세요! 

오늘은 

**"내가 좋아하는 옷 스타일"**

에 대해 이야기해보려고 합니다. 

저는 옷을 통해 자신감을 표현하고 싶어하는 편이에요. 

따라서 옷 스타일은 

**"개성"**

과 

**"편안함"**

을 동시에 추구하는 것을 좋아합니다. 

특히 

**"미니멀리즘"**

과 

**"보헤미안"**

 스타일을 섞어 입는 것을 좋아해요. 

깔끔하고 심플한 디자인의 옷에 

부드러운 소재와 

자연스러운 패턴을 더하여 

자신만의 독특한 스타일을 완성하는 거죠. 

예를 들어, 

* **흰색 면 티셔츠에 

  자수가 있는 

  보헤미안 스타일의 스커트를 매치**

하거나, 

* **검정색 슬랙스에 

  레이스 소재의 블라우스를 

  입는 것**

도 좋아요. 

액세서리로는 

**"진주"**

나 

**"나뭇잎"**

모양의 

액세서리를 활용하여 

자연스러운 분위기를 더하는 것도 

좋아합니다. 

옷을 입는 것은 단순히 몸을 감싸는 것이 아니라 

자신을 표현하고 

세상과 소통하는 

매력적인 방법이라고 생각해요. 

혹시 

**"내가 좋아하는 옷 스타일"**

에 대해 궁금하신 점이 있다면 

댓글로 남겨주세요! 

감사합니다!



In [65]:
input_text='세종대왕이 어느나라 사람이니?'
input_ids=tokenizer(input_text,return_tensors='pt').to('cuda')

In [74]:
outputs=model.generate(input_ids=input_ids.input_ids,
                       return_dict_in_generate=True,
                       output_scores=True,
                       temperature=0.05,
                       top_k=10,
                       top_p= 0.9,
                       do_sample=True
                       )

In [75]:
print(tokenizer.batch_decode(outputs.sequences,skip_special_tokens=True)[0])

세종대왕이 어느나라 사람이니?

세종대왕은 **조선** 사람입니다. 

세종대왕은 고려의 마지막 왕인 태조 이성계의 후손으로, 조선의 4대 왕입니다. 



In [86]:
from transformers import pipeline


In [87]:
pipe=pipeline(model=model, task='text-generation',tokenizer=tokenizer)

In [88]:
input_text='대한민국의 수도는 어디인가요?'

In [91]:
result=pipe(input_text)

In [96]:
print(result[0]['generated_text'][len(input_text):].lstrip())

정답은 **서울**입니다. 



